In [1]:
# Dependancies
import requests
import json
from pprint import pprint
from config import api_key

In [2]:
# Calculations
import dask.dataframe as dd
import pandas as pd
import numpy as np
import re

C:\Users\ashle\anaconda3\envs\dev\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [3]:
# All series variables
series_ids = {
    'unemp_char': 'LAUIM371674000000004',
    'rate_char': 'LAUIM371674000000003',
    'unemp_ral': 'LAUMT373958000000004',
    'rate_ral': 'LAUMT373958000000003',
    'unemp_green': 'LAUMT372466000000004',
    'rate_green': 'LAUMT372466000000003',
    'unemp_win': 'LAUMT374918000000004',
    'rate_win': 'LAUMT374918000000003'
}

start_year = 2015
end_year = 2024
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# For loop sending a request for each series id and saving them as an individual json
for key, series_id in series_ids.items():
    nc_query_url = url + series_id + '?startyear=' + str(start_year) + '&endyear=' + str(end_year) + '&catalog=false'
    response = requests.get(nc_query_url)
    if response.status_code == 200:
        data = response.json()
        with open(key + '.json', 'w') as f:
            json.dump(data, f)
        print(f"Response for {key} saved as {key}.json")
    else:
        print(f"Error fetching data for {key}: Status code {response.status_code}")

Response for unemp_char saved as unemp_char.json
Response for rate_char saved as rate_char.json
Response for unemp_ral saved as unemp_ral.json
Response for rate_ral saved as rate_ral.json
Response for unemp_green saved as unemp_green.json
Response for rate_green saved as rate_green.json
Response for unemp_win saved as unemp_win.json
Response for rate_win saved as rate_win.json


In [4]:
# List of file names
file_names = [
    'unemp_char.json',
    'rate_char.json',
    'unemp_ral.json',
    'rate_ral.json',
    'unemp_green.json',
    'rate_green.json',
    'unemp_win.json',
    'rate_win.json'
]

# Load JSON data from each file into separate variables
for file_name in file_names:
    with open(file_name, 'r') as f:
        globals()[file_name.split('.')[0] + '_data'] = json.load(f)


In [5]:
# print statement to print each json if needed
print(json.dumps(rate_char_data, indent=4))

{
    "status": "REQUEST_SUCCEEDED",
    "responseTime": 198,
    "message": [],
    "Results": {
        "series": [
            {
                "seriesID": "LAUIM371674000000003",
                "data": [
                    {
                        "year": "2024",
                        "period": "M02",
                        "periodName": "February",
                        "latest": "true",
                        "value": "3.6",
                        "footnotes": [
                            {
                                "code": "P",
                                "text": "Preliminary."
                            }
                        ]
                    },
                    {
                        "year": "2024",
                        "period": "M01",
                        "periodName": "January",
                        "value": "3.5",
                        "footnotes": [
                            {}
                        ]
                   

In [31]:
# List of file names
file_names = [
    'Output/unemp_char.json',
    'Output/rate_char.json',
    'Output/unemp_ral.json',
    'Output/rate_ral.json',
    'Output/unemp_green.json',
    'Output/rate_green.json',
    'Output/unemp_win.json',
    'Output/rate_win.json'
]

# List to store individual Pandas DataFrames
dfs = []

# Load JSON data from each file and concatenate into single DataFrame
for file_name in file_names:
    with open(file_name, 'r') as f:
        json_data = json.load(f)
        
    # Extract relevant data from JSON
    series_list = json_data['Results']['series']

    # Flatten the nested structure of the data into a list of dictionaries
    data_list = []
    for series in series_list:
        series_id = series['seriesID']
        for entry in series['data']:
            entry['seriesID'] = series_id
            data_list.append(entry)

    # Create a Pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(data_list)
    dfs.append(df)

# Concatenate all DataFrames into a single Pandas DataFrame
concatenated_df = pd.concat(dfs, ignore_index=True)

# Create a Dask DataFrame from the concatenated Pandas DataFrame
dask_dd = dd.from_pandas(concatenated_df, npartitions=1)

In [32]:
# Checking the dataframe was created pcorrectly
dask_dd.head()

,year,period,periodName,latest,value,footnotes,seriesID
0,2024,M02,February,true,45192,"[{'code': 'P', 'text': 'Preliminary.'}]",LAUIM371674000000004
1,2024,M01,January,NaN,43910,[{}],LAUIM371674000000004
2,2023,M12,December,NaN,38829,"[{'code': 'P', 'text': 'Preliminary.'}]",LAUIM371674000000004
3,2023,M11,November,NaN,40436,[{}],LAUIM371674000000004
4,2023,M10,October,NaN,40193,[{}],LAUIM371674000000004


In [36]:
# Save the full dask dataframe as csv
dask_dd.to_csv('Output/dask_dd.csv', single_file=True, index=False)

['C:\\Data Class\\Project 3\\project3group1\\Output\\dask_dd.csv']

In [37]:
def extract_decimal(value):
    # Define regex pattern to match decimal numbers
    pattern = r'\b\d+\.\d+\b'  # Matches numbers with decimal point
    
    # Search for decimal numbers in the value
    match = re.search(pattern, value)
    
    # If a decimal number is found, return it. Otherwise, return None.
    if match:
        return float(match.group())
    else:
        return None

# Load your data into a Dask DataFrame
clean_dd = dd.read_csv('Output/dask_dd.csv', dtype={'value': 'object'})

# Extract decimal numbers from the 'value' column and create a new column 'rate'
clean_dd['rate'] = clean_dd['value'].apply(extract_decimal, meta=('rate', 'float'))

# Remove decimal numbers from the 'value' column
clean_dd['value'] = clean_dd['value'].astype(str).str.replace(r'\b\d+\.\d+\b', '', regex=True)

# Persist the dataframe if you plan to reuse it
clean_dd = clean_dd.persist()

In [38]:
clean_dd.head()

,year,period,periodName,latest,value,footnotes,seriesID,rate
0,2024,M02,February,True,45192,"[{'code': 'P', 'text': 'Preliminary.'}]",LAUIM371674000000004,NaN
1,2024,M01,January,NaN,43910,[{}],LAUIM371674000000004,NaN
2,2023,M12,December,NaN,38829,"[{'code': 'P', 'text': 'Preliminary.'}]",LAUIM371674000000004,NaN
3,2023,M11,November,NaN,40436,[{}],LAUIM371674000000004,NaN
4,2023,M10,October,NaN,40193,[{}],LAUIM371674000000004,NaN


In [39]:
# Cleaning the dataframe
clean_dd = (clean_dd
            .drop(columns=['period', 'latest', 'footnotes'])
            .rename(columns={'periodName': 'month'})
            [['seriesID', 'month', 'year', 'value', 'rate']]
            .persist())

In [40]:
clean_dd.head()

,seriesID,month,year,value,rate
0,LAUIM371674000000004,February,2024,45192,NaN
1,LAUIM371674000000004,January,2024,43910,NaN
2,LAUIM371674000000004,December,2023,38829,NaN
3,LAUIM371674000000004,November,2023,40436,NaN
4,LAUIM371674000000004,October,2023,40193,NaN


In [41]:
clean_dd.to_csv('Output/clean_dd.csv', single_file=True, index=False)

['C:\\Data Class\\Project 3\\project3group1\\Output\\clean_dd.csv']

In [66]:
char_ids = ['LAUIM371674000000004', 'LAUIM371674000000003']  

char_dd = (clean_dd[clean_dd['seriesID'].isin(char_ids)]
               .drop(columns=['seriesID'])
               .compute()
               .fillna('')
               .astype({'value': str, 'rate': str})
               .groupby(['year', 'month'])
               .agg({'value': ''.join, 'rate': ''.join})
               .reset_index())

In [67]:
char_dd.tail()

,year,month,value,rate
105,2023,November,40436,3.2
106,2023,October,40193,3.2
107,2023,September,38539,3.0
108,2024,February,45192,3.6
109,2024,January,43910,3.5


In [68]:
char_dd.to_csv('Output/char_dd.csv', index=False)

In [69]:
ral_ids = ['LAUMT373958000000004', 'LAUMT373958000000003']  

ral_dd = (clean_dd[clean_dd['seriesID'].isin(ral_ids)]
               .drop(columns=['seriesID'])
               .compute()
               .fillna('')
               .astype({'value': str, 'rate': str})
               .groupby(['year', 'month'])
               .agg({'value': ''.join, 'rate': ''.join})
               .reset_index())

In [70]:
ral_dd.head()

,year,month,value,rate
0,2015,April,28819,4.4
1,2015,August,33120,5.0
2,2015,December,29053,4.4
3,2015,February,30480,4.7
4,2015,January,31855,5.0


In [71]:
ral_dd.to_csv('Output/ral_dd.csv', index=False)

In [72]:
green_ids = ['LAUMT372466000000004', 'LAUMT372466000000003']  

green_dd = (clean_dd[clean_dd['seriesID'].isin(green_ids)]
               .drop(columns=['seriesID'])
               .compute()
               .fillna('')
               .astype({'value': str, 'rate': str})
               .groupby(['year', 'month'])
               .agg({'value': ''.join, 'rate': ''.join})
               .reset_index())

In [73]:
green_dd.head()

,year,month,value,rate
0,2015,April,19930,5.5
1,2015,August,22314,6.1
2,2015,December,19395,5.4
3,2015,February,21209,5.9
4,2015,January,22125,6.1


In [74]:
green_dd.to_csv('Output/green_dd.csv', index=False)

In [63]:
win_ids = ['LAUMT374918000000004', 'LAUMT374918000000003'] 

win_dd = (clean_dd[clean_dd['seriesID'].isin(win_ids)]
               .drop(columns=['seriesID'])
               .compute()
               .fillna('')
               .astype({'value': str, 'rate': str})
               .groupby(['year', 'month'])
               .agg({'value': ''.join, 'rate': ''.join})
               .reset_index())

In [64]:
win_dd.head()

,year,month,value,rate
0,2015,April,15827,5.0
1,2015,August,17890,5.7
2,2015,December,15514,4.9
3,2015,February,16958,5.4
4,2015,January,17733,5.6


In [65]:
win_dd.to_csv('Output/win_dd.csv', index=False)